In [ ]:
model_name = 'DeepPavlov/rubert-base-cased-sentence'
BATCH_SIZE = 32
SIMILARITY_THRESHOLD = 0.85

# Imports

In [ ]:
! pip install -q transformers
! pip install -q persist-queue

     |████████████████████████████████| 1.8MB 8.1MB/s 
     |████████████████████████████████| 890kB 43.6MB/s 
     |████████████████████████████████| 3.2MB 43.9MB/s 


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torchtext import data
from transformers import AutoModel, AutoTokenizer
from persistqueue import Queue
from tqdm.auto import tqdm

import pickle
import warnings
from os import path, remove
from shutil import copyfile, rmtree
from typing import Callable, Optional, List, Tuple, Collection, Union
from copy import copy
from itertools import chain

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
ROOT_DIR = ''

In [ ]:
# Source https://github.com/pytorch/pytorch/issues/11202
def cosine_similarity(x1, x2=None, eps=1e-8):
    x2 = x1 if x2 is None else x2
    w1 = x1.norm(p=2, dim=1, keepdim=True)
    w2 = w1 if x2 is x1 else x2.norm(p=2, dim=1, keepdim=True)
    return torch.mm(x1, x2.t()) / (w1 * w2.t()).clamp(min=eps)

# Pairs distance heuristic

In [ ]:
! pip install -q "textdistance[extras]"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 14.7MB/s 
     |████████████████████████████████| 102kB 11.0MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 


In [ ]:
from textdistance import levenshtein, bag
from nltk import word_tokenize, wordpunct_tokenize
from nltk import download as nltk_download
nltk_download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
class QuestionPairBase(object):
    """
    Class for question pairs
    Two pairs are equal, if they have small Levenshtein distance
    """
    def __init__(self,
                 left_id: int, left_text: str,
                 right_id: int, right_text: str,
                 similarity: float):
        self.left_id = left_id
        self.left_text = left_text
        self.right_id = right_id
        self.right_text = right_text
        self.similarity = similarity

    @staticmethod
    def _question_eq(left: str, right: str) -> bool:
        return left == right

    def __eq__(self, other) -> bool:
        left_left_eq = self._question_eq(self.left_text, other.left_text)
        right_right_eq = self._question_eq(self.right_text, other.right_text)

        if left_left_eq and right_right_eq:
            return True

        left_right_eq = self._question_eq(self.left_text, other.right_text)
        right_left_eq = self._question_eq(self.right_text, other.left_text)
        if left_right_eq and right_left_eq:
            return True
        else:
            return False

    def __ne__(self, other) -> bool:
        return not self.__eq__(other)

    def __hash__(self):
        # order agnostic hash
        return self.left_text.__hash__() ^ self.right_text.__hash__()
    

In [ ]:
def create_heuristic_comparator(eq_heuristic: Callable[[str, str], bool]):
    class QuestionPair(QuestionPairBase):
        @staticmethod
        def _question_eq(left: str, right: str) -> bool:
            return eq_heuristic(left, right)

    return QuestionPair

In [ ]:
LevCharCmp = create_heuristic_comparator(
    lambda s1, s2: levenshtein(s1, s2) <= 5
    )

def cmp_heuristic(left: str, right: str) -> bool:
    # Compare two questions, using several heuristics

    # compare in lower case
    left = left.lower()
    right = right.lower()

    if levenshtein(left, right) <= 4:
        # Two questions are very close (as symbol sequence)
        return True

    tok_left = word_tokenize(left)
    tok_right = word_tokenize(right)

    bow_dist = bag(tok_left, tok_right)
    if bow_dist > 2:
        # Two questions contains different words
        return False
    
    lev_tok_dist = levenshtein(tok_left, tok_right)
    if lev_tok_dist <= 2:
        # Too close tokens sequence.
        # Two questions are equal with respect of some noise
        return True

    # May be a transposition or easy but meaningful paraphrase
    return False

QuestionPair = create_heuristic_comparator(cmp_heuristic)

In [ ]:
def construct_similarity_dataframe(
    it: Collection[Tuple[int, int, float]]
    ) -> pd.DataFrame:
    
    questions_df = pd.read_csv(ROOT_DIR + 'questions.csv', sep=';')

    pairs = set()

    for left_id, right_id, sim_val in tqdm(it, unit='pair'):
        left_text = questions_df.loc[questions_df['id'] == left_id, 'text'].item()
        right_text = questions_df.loc[questions_df['id'] == right_id, 'text'].item()

        # There are some pairs with too similar text
        # Ignore too close (Levenshtein distance) pairs

        if cmp_heuristic(left_text, right_text):
            continue

        pair_obj = QuestionPair(
            left_id,
            left_text,
            right_id,
            right_text,
            sim_val
        )
        pairs.add(pair_obj)

    n_pairs = len(pairs)
    sim_df = pd.DataFrame({
        'left_id': [-1] * n_pairs,
        'left_text': [''] * n_pairs,
        'left_url': [''] * n_pairs,
        'right_id': [-1] * n_pairs,
        'right_text': [''] * n_pairs,
        'right_url': [''] * n_pairs,
        'similarity': [0.] * n_pairs
        })

    for i, p in enumerate(pairs):
        left_url = questions_df.loc[questions_df['id'] == p.left_id, 'url'].item()
        right_url = questions_df.loc[questions_df['id'] == p.right_id, 'url'].item()
        sim_df.at[i, 'left_url'] = left_url
        sim_df.at[i, 'right_url'] = right_url

        sim_df.at[i, 'left_text'] = p.left_text
        sim_df.at[i, 'right_text'] = p.right_text
        sim_df.at[i, 'left_id'] = p.left_id
        sim_df.at[i, 'right_id'] = p.right_id
        sim_df.at[i, 'similarity'] = p.similarity

    return sim_df

In [ ]:
def sim_dataframe_from_queue(queue: Queue) -> pd.DataFrame:
    class QueueIter(object):
        def __init__(self, queue: Queue):
            self.queue = queue
            self.size = self.queue.qsize()
            self.remain = copy(self.size)

        def __len__(self):
            return self.size

        def __iter__(self):
            return self

        def __next__(self):
            if self.remain > 0:
                self.remain -= 1
                return self.queue.get()
            else:
                raise StopIteration

    seq_obj = QueueIter(queue)
    return construct_similarity_dataframe(seq_obj)

# BERT embeddings

## Make up dataset

In [ ]:
class HuggingFaceField(data.Field):
    def __init__(self, tokenizer):
        super().__init__(
            tokenize=tokenizer.tokenize,
            use_vocab=False,
            pad_token=tokenizer.pad_token,
            init_token=tokenizer.cls_token
        )
        self.tokenizer = tokenizer

    def numericalize(self, arr, device):
        arr = [self.tokenizer.convert_tokens_to_ids(x) for x in arr]
        return torch.LongTensor(arr).to(device)

class LabelField(data.Field):
    def __init__(self):
        super().__init__(
            use_vocab=False,
            sequential=False,
            tokenize=lambda x: x)
        
    def numericalize(self, arr, device):
        arr = [int(item) for item in arr]
        return torch.LongTensor(arr).to(device)        

Loading CSV file with questions. It supposed that file contains 'id' (the question unique id in dataset) and 'text' (text of the question) columns.

Supposed that `ROOT_DIR` variable stores path to directory with data.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
pad_index = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
unk_index = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)
cls_index = tokenizer.convert_tokens_to_ids(tokenizer.cls_token)
sep_index = tokenizer.convert_tokens_to_ids(tokenizer.sep_token)

Remove already processed items from dataset

In [ ]:
if path.isfile(ROOT_DIR + 'done_ids.txt'):
    with open(ROOT_DIR + 'done_ids.txt') as f:
        done = set(int(v) for v in f.read().rstrip().split())

    df = pd.read_csv(ROOT_DIR + 'questions.csv', sep=';')
    df = df[~df['id'].isin(done)]
    df.to_csv('./questions.csv', index=False, sep=';')
else:
    copyfile(ROOT_DIR + 'questions.csv', './questions.csv')
    done = set()

In [ ]:
TEXT = HuggingFaceField(tokenizer)
ID = LabelField()
dataset = data.TabularDataset(
    'questions.csv',
    'CSV',
    {'text': ('text', TEXT), 'id': ('id', ID)},
    skip_header=False,
    csv_reader_params={'delimiter': ';'}
)

iterator = data.BucketIterator(
    dataset,
    BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    device=device,
    shuffle=False
)

Initializing file to store intermediate data

In [ ]:
embed_queue = Queue(ROOT_DIR + 'embed.queue', autosave=False)

## Evaluation

In [ ]:
model = AutoModel.from_pretrained(model_name).to(device)

In [ ]:
with open(ROOT_DIR + 'done_ids.txt', 'a') as done_f:
    with torch.no_grad():
        model.eval()
        for batch in tqdm(iterator, unit='batch'):
            tokens = batch.text
            mask = (tokens != pad_index).float()

            output = model(
                tokens,
                attention_mask=mask,
                output_hidden_states=False,
                return_dict=True
            )

            token_embeddings = output['last_hidden_state']

            # Mean-pooling for sentence embedding.
            # The same maner as in original Sentence-BERT work
            sent_embeddings = torch.sum(token_embeddings * mask.unsqueeze(-1),
                                        dim=1)
            mask_sum = torch.sum(mask, dim=1, keepdim=True)
            sent_embeddings = sent_embeddings / mask_sum

            # Saving embeddings to queue and add processed id to file
            for (sent_id, emb) in zip(batch.id.cpu(), sent_embeddings.cpu()):
                sent_id = int(sent_id)
                embed_queue.put((sent_id, emb))

                done_f.write(str(sent_id) + ' ')
                done.add(sent_id)

In [ ]:
questions_embeddings = {}
q_size = embed_queue.qsize()
for _ in range(q_size):
    sent_id, emb_vec = embed_queue.get()
    questions_embeddings[sent_id] = emb_vec.numpy()

with open(ROOT_DIR + 'embeddings.pkl', 'wb') as f:
    pickle.dump(questions_embeddings, f)

assert embed_queue.empty()
assert len(questions_embeddings) == q_size

del embed_queue

# Embeddings similarity

Explicit similarity matrix wiil be too huge to fit the memory.

To overcome this limit, all embeddings will be splitted on batches. Similarity matrix will be calculated between every batch pair.

In [ ]:
if not 'questions_embeddings' in locals() and not 'questions_embeddings' in globals():
    with open(ROOT_DIR + 'embeddings.pkl', 'rb') as f:
        questions_embeddings = pickle.load(f)

# Sort dataset ids to reproduce order, if calculation is interrupted
dataset_ids = np.array(sorted(questions_embeddings.keys()))
emb_dim = next(iter(questions_embeddings.values())).shape[0]
embeddings = np.empty((len(questions_embeddings), emb_dim), dtype=np.float)

for i, sent_id in enumerate(dataset_ids):
    embeddings[i] = questions_embeddings[sent_id]

del questions_embeddings

In [ ]:
SIM_BATCH_SIZE = 1024

In [ ]:
if 'model' in locals() or 'model' in globals():
    del model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [ ]:
embeddings = torch.from_numpy(embeddings).detach().to(device)

In [ ]:
sim_pairs_queue = Queue(ROOT_DIR + 'sim.queue', autosave=False)

batch_start_idx = list(range(0, embeddings.size(0), SIM_BATCH_SIZE))

# Load last finished indexes
if not path.isfile(ROOT_DIR + 'sim_last_iter.txt'):
    with open(ROOT_DIR + 'sim_last_iter.txt', 'w') as f:
        f.write('-1 ' + str(len(batch_start_idx) - 1) + ' 0\n')

with open(ROOT_DIR + 'sim_last_iter.txt') as f:
    last_i, last_j, n_found = [int(v) for v in f.readline().rstrip().split()]

if last_j + 1 >= len(batch_start_idx):
    start_i = last_i + 1
    start_j = start_i
else:
    start_i = last_i
    start_j = last_j + 1

start_i = max(0, start_i)
start_j = max(start_j, start_i)

# Calculate info for progress bar
total_pairs = len(batch_start_idx) * (len(batch_start_idx) + 1) // 2
init_pairs = start_i * (2 * len(batch_start_idx) - start_i + 1) // 2
init_pairs += start_j - start_i

with open(ROOT_DIR + 'sim_last_iter.txt', 'r+') as last_it_f, torch.no_grad():
    with tqdm(total=total_pairs, unit='batch pair', initial=init_pairs) as pbar:
        for i in range(start_i, len(batch_start_idx)):
            batch1_start = batch_start_idx[i]
            batch1 = embeddings[batch1_start: batch1_start + SIM_BATCH_SIZE]

            for j in range(max(i, start_j), len(batch_start_idx)):
                start_j = -1

                batch2_start = batch_start_idx[j]
                batch2 = embeddings[batch2_start: batch2_start + SIM_BATCH_SIZE]

                sim_matrix = cosine_similarity(batch1, batch2).cpu()
                if i == j:
                    sim_matrix = torch.triu(sim_matrix, diagonal=1)

                # Using calculated batch-batch similarity matrix 
                # recover corresponding ids in dataset
                idx = torch.where(sim_matrix >= SIMILARITY_THRESHOLD)
                for batch1_shift, batch2_shift in zip(*idx):
                    left_id = batch1_start + batch1_shift.item()
                    right_id = batch2_start + batch2_shift.item()

                    left_id = dataset_ids[left_id].item()
                    right_id = dataset_ids[right_id].item()
                    sim_val = sim_matrix[batch1_shift, batch2_shift].item()

                    # Save close pair to queue
                    sim_pairs_queue.put((left_id, right_id, sim_val))
                    n_found += 1

                # Using try clase to log progress, 
                # even if execution was interuppted by Ctrl-C
                try:
                    pass
                except Exception as e:
                    raise e
                finally:
                    # Update progress
                    last_it_f.seek(0)
                    last_it_f.truncate(0)
                    last_it_f.write(f'{i} {j} {n_found}\n')
                    pbar.update()
                pbar.set_postfix({'similar pairs found': n_found})

Save the resulting pairs in CSV table

In [ ]:
if 'sim_pairs_queue' not in locals() and 'sim_pairs_queue' not in globals(): 
    sim_pairs_queue = Queue(ROOT_DIR + 'sim.queue', autosave=False)
    print(sim_pairs_queue.qsize())

Default tempdir '/tmp/tmpqxhqy0hj' is not on the same filesystem with queue path '/content/drive/MyDrive/ParaPhrase/Unsupervised embeddings/sim.queue',defaulting to '/content/drive/MyDrive/ParaPhrase/Unsupervised embeddings/sim.queue'.


254748


In [ ]:
sim_df = sim_dataframe_from_queue(sim_pairs_queue)
sim_df = sim_df.sort_values(by=['similarity'], ascending=False)

sim_df.to_csv(ROOT_DIR + 'sim.csv', sep=';', index=False)

# Similarity bins

In this section pairs with different similarity values are collected.

In [ ]:
SIM_BATCH_SIZE = 1024
BIN_SIZE = 50
CROWD_OUT_PROB = 0.1

In [ ]:
def calc_bins(
    embeddings: torch.Tensor,
    dataset_ids: torch.LongTensor
    ) -> List[List[Tuple[int, int, float]]]:

    bins = [[] for _ in range(10)]
    batch_start_idx = list(range(0, embeddings.size(0), SIM_BATCH_SIZE))

    # Collect more pair then necessary, because of possible identical pairs 
    bin_size = BIN_SIZE * 3 // 2

    # Calculate info for progress bar
    total_pairs = len(batch_start_idx) * (len(batch_start_idx) + 1) // 2
    pairs_pbar = tqdm(total=total_pairs, unit='batch', position=1, desc='All pairs processed')
    bins_pbar = tqdm(total=100, position=0, desc="Bins progress")

    for i in range(len(batch_start_idx)):
        batch1_start = batch_start_idx[i]
        batch1 = embeddings[batch1_start: batch1_start + SIM_BATCH_SIZE]

        for j in range(i, len(batch_start_idx)):
            batch2_start = batch_start_idx[j]
            batch2 = embeddings[batch2_start: batch2_start + SIM_BATCH_SIZE]

            sim_matrix = cosine_similarity(batch1, batch2)
            if i == j:
                sim_matrix = torch.triu(sim_matrix, diagonal=1)

            # Extend bins
            min_bin_size = bin_size + 1
            for bin_i in range(10):
                bin_sim_min = 0.1 * bin_i
                bin_sim_max = bin_sim_min + 0.1

                idx = torch.where(
                    (sim_matrix > bin_sim_min)
                    &
                    (sim_matrix <= bin_sim_max)
                    )
                
                for batch1_shift, batch2_shift in zip(*idx):
                    # Using calculated batch-batch similarity matrix 
                    # recover corresponding ids in dataset
                    left_id = batch1_start + batch1_shift.item()
                    right_id = batch2_start + batch2_shift.item()

                    left_id = dataset_ids[left_id].item()
                    right_id = dataset_ids[right_id].item()
                    sim_val = sim_matrix[batch1_shift, batch2_shift].item()
                    if len(bins[bin_i]) < bin_size:
                        bins[bin_i].append((left_id, right_id, sim_val))
                    elif np.random.random() <= CROWD_OUT_PROB:
                        # Bin is completely filled
                        k = np.random.randint(bin_size)
                        bins[bin_i][k] = (left_id, right_id, sim_val)
                
                min_bin_size = min(min_bin_size, len(bins[bin_i]))

            pairs_pbar.update()
            bins_pbar.update(100 * min_bin_size // bin_size - bins_pbar.n)
            if min_bin_size >= bin_size:
                pairs_pbar.close()
                bins_pbar.close()
                return bins
    
    pairs_pbar.close()
    bins_pbar.close()
    return bins

In [ ]:
if not 'questions_embeddings' in locals() and not 'questions_embeddings' in globals():
    with open(ROOT_DIR + 'embeddings.pkl', 'rb') as f:
        questions_embeddings = pickle.load(f)

# Sort dataset ids to reproduce order, if calculation is interrupted
dataset_ids = np.array(sorted(questions_embeddings.keys()))
emb_dim = next(iter(questions_embeddings.values())).shape[0]
embeddings = np.empty((len(questions_embeddings), emb_dim), dtype=np.float)

for i, sent_id in enumerate(dataset_ids):
    embeddings[i] = questions_embeddings[sent_id]

del questions_embeddings

In [ ]:
embeddings = torch.from_numpy(embeddings).detach().to(device)

In [ ]:
idx = torch.randperm(embeddings.shape[0])
bins = calc_bins(embeddings[idx], dataset_ids[idx])

In [ ]:
ext_bins_df = construct_similarity_dataframe(chain.from_iterable(bins))

for i in range(10):
    bin_sim_min = 0.1 * i
    bin_sim_max = bin_sim_min + 0.1

    bin = ext_bins_df.loc[(ext_bins_df.similarity > bin_sim_min) & (ext_bins_df.similarity <= bin_sim_max)]
    if len(bin) < BIN_SIZE:
        warnings.warn(f'i th bin is smaller then required size')
    else:
        bin = bin.sample(BIN_SIZE)

    bin['bin'] = i
    if i == 0:
        bins_df = bin
    else:
        bins_df = pd.concat([bins_df, bin])
bins_df = bins_df.sort_values(by=['bin'])
bins_df.to_csv(ROOT_DIR + 'calibration_bins.csv', index=False, sep=';')

# Cleanup

In [ ]:
rmtree(ROOT_DIR + 'embed.queue')
remove(ROOT_DIR + 'done_ids.txt')

rmtree(ROOT_DIR + 'sim.queue')
remove(ROOT_DIR + 'sim_last_iter.txt')